# Рекомендация тарифов

В вашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — вы её уже сделали.

Постройте модель с максимально большим значением *accuracy*. Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75. Проверьте *accuracy* на тестовой выборке самостоятельно.

## Откройте и изучите файл

In [ ]:
import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.metrics import accuracy_score 
from sklearn.model_selection import train_test_split 
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics import mean_squared_error
from sklearn.metrics import f1_score
from sklearn.metrics import recall_score
from sklearn.metrics import precision_score
from sklearn.dummy import DummyClassifier

In [ ]:
df = pd.read_csv('/datasets/users_behavior.csv')

In [ ]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [ ]:
df.head()

,calls,minutes,messages,mb_used,is_ultra
0,40.0,311.90,83.0,19915.42,0
1,85.0,516.75,56.0,22696.96,0
2,77.0,467.66,86.0,21060.45,0
3,106.0,745.53,81.0,8437.39,1
4,66.0,418.74,1.0,14502.75,0


In [ ]:
df.is_ultra.value_counts( )

0    2229
1     985
Name: is_ultra, dtype: int64

## Разбейте данные на выборки

In [ ]:
df_train, df_test = train_test_split(df, test_size=0.20, random_state=12345)

In [ ]:
df_train,df_valid = train_test_split(df, test_size=0.20, random_state=12345)

## Исследуйте модели

Ошибки

In [ ]:
def error_count(answers, predictions):
    count = 0
    for i in range(len(answers)):
        if answers[i] != predictions[i]:
            count += 1
    return count

Точность

In [ ]:
def accuracy(answers, predictions):
    accur = 0
    for i in range(len(answers)):
        if answers[i] != predictions[i]:
            accur += 1
    accur = (len(answers) - accur) / len(answers)
    return accur

### Классификатор дерева решений (DecisionTreeClassifier)

Определим признаки и цель

In [ ]:
#Обучающаяся
features_train = df_train.drop(['is_ultra'], axis=1)
target_train = df_train['is_ultra']
#Тестовая
features_test = df_test.drop(['is_ultra'], axis=1)
target_test = df_test['is_ultra']
#Валидная
features_valid = df_valid.drop(['is_ultra'], axis=1)
target_valid = df_valid['is_ultra']


Тренеруем модель

In [ ]:
model = DecisionTreeClassifier(random_state=12345)
model.fit(features_train, target_train)

DecisionTreeClassifier(random_state=12345)

### Проверьте модель на тестовой выборке

In [ ]:
test = model.predict(features_test)

In [ ]:
print("Ошибок:", error_count(target_test.values, test))
print("Accuracy:", accuracy(target_test.values, test))

Ошибок: 181
Accuracy: 0.71850699844479


На тестовой модели 181 ошибка. Нужно протестировать с разной глубиной depth

In [ ]:
best_model = None
best_result = 0
depth = None
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # обучите модель с заданной глубиной дерева
    model.fit(features_train,target_train) # обучите модель
    predictions = model.predict(features_train) # получите предсказания модели
    result = accuracy_score(target_train, predictions) # посчитайте качество модели
    if result > best_result:
        best_model = model
        best_result = result
        dep = depth
print("Accuracy лучшей модели:", best_result,'глубина:',depth)


Accuracy лучшей модели: 0.8144690781796966 глубина: 5


Лучшая модель на тестовой выборке на 5 глубине.

### Валидация

In [ ]:
best_model = None
best_result = 0
best_rmse = 1000
depth = None
for depth in range(1, 6):
    model = DecisionTreeClassifier(random_state=12345, max_depth=depth) # обучите модель с заданной глубиной дерева
    model.fit(features_valid,target_valid) # обучите модель
    predictions_valid = model.predict(features_valid) # получите предсказания модели
    result = accuracy_score(target_valid, predictions_valid) # посчитайте качество модели
    rmsee = mean_squared_error(target_valid, predictions_valid)**0.5
    if result > best_result:
        best_model = model
        best_result = result
        dep = depth
    if rmsee < best_rmse:
        best_rmse = rmsee
print("Accuracy лучшей модели:", best_result,'глубина:',depth)
print('RMSE:',best_rmse)

Accuracy лучшей модели: 0.8444790046656299 глубина: 5
RMSE: 0.3943615033625495


### Вывод

При модели DecisionTreeClassifier лучшей моделью является модель с глубиной в 5.

### Случайный лес (RandomForestClassifier)

In [ ]:
best_model = None
best_result = 0
est = None
best_rmse = 1000
for est in range(100, 1001, 100):
    model = RandomForestClassifier(random_state=12345, n_estimators=est) # обучите модель с заданным количеством деревьев
    model.fit(features_train,target_train) # обучите модель на тренировочной выборке
    predict = model.predict(features_train)
    predict_valid = model.predict(features_valid)
    result = model.score(features_valid,target_valid) # посчитайте качество модели на валидационной выборке
    rmsee = mean_squared_error(target_valid, predict_valid)**0.5
    if result > best_result:
        best_model = model # сохраните наилучшую модель
        best_result = result   # сохраните наилучшее значение метрики accuracy на валидационных данных
    if rmsee < best_rmse:
        best_rmse = rmsee
print("Accuracy наилучшей модели на валидационной выборке:", best_result,'количество оценщиков:',est)
print('RMSE:',best_rmse)

Accuracy наилучшей модели на валидационной выборке: 0.7822706065318819 количество оценщиков: 1000
RMSE: 0.46661482345519006


### Логистическая регрессия (LogisticRegression)

In [ ]:
model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=1000) 
model.fit(features_train, target_train) 
predict_valid = model.predict(features_valid) 
print('Accuracy наилучшей модели на валидационной выборке::',model.score(features_train, target_train))
RMSE = mean_squared_error(target_valid, predict_valid)**0.5
print('RMSE',RMSE)

Accuracy наилучшей модели на валидационной выборке:: 0.7467911318553092
RMSE 0.492557359828901


## (бонус) Проверьте модели на адекватность

Контрастная модель

In [ ]:
# target_train_zeros = df_train['is_ultra']
# target_allzero_predicion_constant = pd.Series ([0] * len(target_train_zeros))

# model = DecisionTreeClassifier(random_state=12345) #  инициализируйте модель с наилучшим значением метрики RMSE
# model.fit(features_train,target_allzero_predicion_constant) # обучите модель на тренировочной выборке
# predictions_valid = model.predict(features_valid)
# f1_score(target_train_zeros.values,predictions_valid)

In [ ]:
dc_mf = DummyClassifier(strategy="most_frequent") 
dc_mf.fit(features_train, target_train)
naive_forecast = dc_mf.score(features_test, target_test)
print(f'Наивный прогноз: {naive_forecast}. Наш прогноз: 0.8444790046656299')

Наивный прогноз: 0.6951788491446346. Наш прогноз: 0.8444790046656299


In [ ]:
# model = DecisionTreeClassifier(random_state=12345) #  инициализируйте модель с наилучшим значением метрики RMSE
# model.fit(features_train,target_train) # обучите модель на тренировочной выборке
# predictions_valid = model.predict(features_valid) # получите предсказания модели на валидационной выборке
# result = mean_squared_error(target_valid, predictions_valid)**0.5# посчитайте значение метрики RMSE на валидационной выборке
# print("RMSE наилучшей модели на валидационной выборке:", result)

## Общий вывод

Лучшая модель для данной задачи это Классификатор дерева решений (DecisionTreeClassifier) так как скорость тут не в приоритете и точность наиболее высокая среди всех моделей. RMSE наилучшей модели на валидационной выборке: 0.5305591404878536

# <font color='orange'>Общее впечатление</font>
* Этот проект выполнен очень хорошо
* Достигнута необходимая метрика accuracy
* Видно, что ты уверенно кодишь и понимаешь базу машинного обучения
* Молодец, что структурируешь ноутбук, приятно проверять такие работы
* Проект можно уже принимать, но я хочу, чтобы ты исправил дополнительное задание
* Можешь вернуть мне проект без правок - я его приму
* Также я оставил несколько советов. Исправь их если останется время.
* Жду новую версию проекта 👋

## Чек-лист готовности проекта

- [x] Jupyter Notebook открыт
- [x] Весь код исполняется без ошибок
- [x] Ячейки с кодом расположены в порядке исполнения
- [x] Выполнено задание 1: данные загружены и изучены
- [x] Выполнено задание 2: данные разбиты на три выборки
- [X] Выполнено задание 3: проведено исследование моделей
    - [X] Рассмотрено больше одной модели
    - [X] Рассмотрено хотя бы 3 значения гипепараметров для какой-нибудь модели
    - [X] Написаны выводы по результатам исследования
- [X] Выполнено задание 3: Проведено тестирование
- [X] Удалось достичь accuracy не меньше 0.75
